## Data Prep Merge

In [ ]:
import pandas as pd

# Mergen: Umsatzdaten mit Kieler Woche (Left Join)
merged_df = df_umsatz.merge(df_kiwo, on='Datum', how='left')

# Mergen: Das Ergebnis mit Wetterdaten (Left Join)
merged_df = merged_df.merge(df_wetter, on='Datum', how='left')

# Ergebnis anzeigen
print(merged_df.head())

# Anzeigen der Spaltennamen (Variablen) im gemergten DataFrame
print(merged_df.columns)

# save the merged df as df_merged_umsatz_kiwo_wetter
merged_df.to_csv("df_merged_umsatz_kiwo_wetter.csv", index=False)